<a href="https://colab.research.google.com/github/neerajgosai19/data-science/blob/master/Restricted_Boltzmann_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
with urllib.request.urlopen("http://deeplearning.net/tutorial/code/utils.py") as url:
  response = url.read()
target = open('utils.py' , 'w')
target.write(response.decode('utf-8'))
target.close()

In [2]:
!pip install grpcio
!pip install tensorflow==2.2.0
!pip install pillow


     |████████████████████████████████| 516.2MB 25kB/s 
     |████████████████████████████████| 460kB 37.4MB/s 
     |████████████████████████████████| 3.0MB 36.9MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [3]:
import tensorflow as tf
from IPython.display import Markdown , display
def printmd(string):
  display(Markdown('# < span style = "color:blue">' + string + '</span>'))
if not tf.__version__ == '2.2.0':
  printmd('<<<<<!!!!! ERROR !!!! please upgrade to TensorFlow 2.2.0, or restart your Kernel (Kernel->Restart & Clear Output)>>>>>')

In [4]:
import tensorflow as tf
import numpy as np
from PIL import Image
from utils import tile_raster_images
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
v_bias = tf.Variable(tf.zeros([7]), tf.float32)
h_bias = tf.Variable(tf.zeros([2]), tf.float32)

In [6]:
W = tf.constant(np.random.normal(loc= 0.0 , scale = 1.0 , size=(7,2)).astype(np.float32))

Before we go further, let's look at a toy example for one case out of all input. Assume that we have a trained RBM, and a very simple input vector, such as [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]

In [7]:
X = tf.constant([[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]],tf.float32)
v_state = X
print('v_State input is   :' , v_state)

v_State input is   : tf.Tensor([[1. 0. 0. 1. 0. 0. 0.]], shape=(1, 7), dtype=float32)


In [8]:
h_bias = tf.constant([0.1, 0.1])
print('hb : ' , h_bias)
print( 'W' , W)

hb :  tf.Tensor([0.1 0.1], shape=(2,), dtype=float32)
W tf.Tensor(
[[ 1.5673054  -2.1570647 ]
 [-0.2501203  -0.2890857 ]
 [-0.99101835 -0.00397976]
 [ 0.5386768   0.75148547]
 [ 0.54450536  0.5033698 ]
 [-0.02519927 -0.74991506]
 [-0.588251    0.5456763 ]], shape=(7, 2), dtype=float32)


In [9]:
# Calculate the probabilities of turning the hidden units on:
h_prob = tf.nn.sigmoid(tf.matmul(v_state, W) + h_bias)
print("p(h|v):" , h_prob)

p(h|v): tf.Tensor([[0.9007855  0.21322756]], shape=(1, 2), dtype=float32)


In [10]:
# Draw samples from the distribution:
h_state = tf.nn.relu(tf.sign(h_prob - tf.random.uniform(tf.shape(h_prob))))
print("h0 states: " , h_state )

h0 states:  tf.Tensor([[1. 0.]], shape=(1, 2), dtype=float32)


phase 2) Backward Pass (Reconstruction): The RBM reconstructs data by making several forward and backward passes between the visible and hidden layers.

In [11]:
vb = tf.constant([0.1, 0.2, 0.1, 0.1, 0.1, 0.2, 0.1])
print ("b   :"   , vb)

b   : tf.Tensor([0.1 0.2 0.1 0.1 0.1 0.2 0.1], shape=(7,), dtype=float32)


In [12]:
v_prob = tf.nn.sigmoid(tf.matmul(h_state , tf.transpose(W)) + vb)
print("p(vi∣h)" , v_prob)

p(vi∣h) tf.Tensor(
[[0.8412162  0.48747256 0.29089972 0.6544543  0.6557712  0.54358923
  0.38030568]], shape=(1, 7), dtype=float32)


 with probability  𝑝(ℎ𝑗|𝑚𝑎𝑡ℎ𝑏𝑓𝑣) , and turn it off with probability  1−𝑝(ℎ𝑗|𝑚𝑎𝑡ℎ𝑏𝑓𝑣)  by generating a uniform random number vector  𝑚𝑎𝑡ℎ𝑏𝑓𝑥𝑖 , and comparing it to the activation probability as

If  𝜉𝑗>𝑝(ℎ𝑗|𝐯) , then  ℎ𝑗=1 , else  ℎ𝑗=0 .

In [13]:
v_state = tf.nn.relu(tf.sign(v_prob - tf.random.uniform(tf.shape(v_prob))))
print("v probability states: "  , v_state )

v probability states:  tf.Tensor([[1. 0. 1. 0. 0. 1. 1.]], shape=(1, 7), dtype=float32)


In [14]:
inp = X
print("input X   :" ,  inp.numpy())
print("probablity vector:" , v_prob[0].numpy())
v_probability = 1
for elm , p in zip(inp[0],v_prob[0]):
  if elm == 1:
    v_probability *= p
  else:
    v_probability *= (1-p)



input X   : [[1. 0. 0. 1. 0. 0. 0.]]
probablity vector: [0.8412162  0.48747256 0.29089972 0.6544543  0.6557712  0.54358923
 0.38030568]


In [15]:
#loading training and test data
mnist = tf.keras.datasets.mnist
(trX , trY) , (teX , teY) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [16]:
from tensorflow.keras.layers import Flatten
flatten = Flatten(dtype = 'float32')
trX = flatten(trX/255.0)
trY = flatten(trY/255.0)

In [17]:
#MNIST images have 784 pixels, so the visible layer must have 784 input nodes. For our case, we'll use 50 nodes in the hidden layer, so i = 50.
vb = tf.Variable(tf.zeros([784]), dtype= tf.float32)
hb = tf.Variable(tf.zeros([50]) , dtype= tf.float32)

In [18]:
W = tf.Variable(tf.zeros([784,50]) , dtype= tf.float32)
visi = tf.Variable(tf.zeros([784]) , dtype= tf.float32)
tf.matmul([visi ], W)

<tf.Tensor: shape=(1, 50), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]], dtype=float32)>

In [19]:
hb_prob = tf.nn.sigmoid(tf.matmul([visi] , W )+ hb)
print("h0_state shape:" , tf.shape(hb_prob))

h0_state shape: tf.Tensor([ 1 50], shape=(2,), dtype=int32)


In [20]:
#defining a function to return only the generated hidden states 
def hidden_layer(visi , W , hb):
  hid_prob = tf.nn.sigmoid(tf.matmul([visi ], W) + hb)  #probabilities of the hidden units
  hid_State = tf.nn.relu(tf.sign(hid_prob - tf.random.uniform(tf.shape(hid_prob))))
  return hid_State
hid_State = hidden_layer(visi , W , hb)
print("first 15 hidden states  : "   , hid_State[0][0:15])

first 15 hidden states  :  tf.Tensor([0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0.], shape=(15,), dtype=float32)


In [21]:
#Now, we define reconstruction part:
def reconstructed_output(hid_State , W , hb):
  v1_prob = tf.nn.sigmoid(tf.matmul(hid_State ,tf.transpose(W)) + vb)
  v1_State = tf.nn.relu(tf.sign(v1_prob - tf.random.uniform(tf.shape(v1_prob))))
  return v1_State
v1_State = reconstructed_output(hid_State , W , hb)
print("hidden state shape: " , hid_State.shape )
print("visi state shape:  ", visi.shape )
print("v1 state shape:  ",  v1_State.shape  )

hidden state shape:  (1, 50)
visi state shape:   (784,)
v1 state shape:   (1, 784)


Calculate error:
In each epoch, we compute the "error" as a sum of the squared difference between step 1 and step n, e.g the error shows the difference between the data and its reconstruction.

In [22]:
def error_Calculate(visi , v1_State):
  return tf.reduce_mean(tf.square(visi - v1_State))
err = tf.reduce_mean(tf.square(visi - v1_State))
print("the error is :  " , err.numpy())


the error is :   0.51403064


In [23]:
h1_prob = tf.nn.sigmoid(tf.matmul([v1_State ], W ) +  hb )
h1_State = tf.nn.relu(tf.sign(h1_prob - tf.random.uniform(tf.shape(h1_prob))))
print(" error :  "   , v1_State , visi )

 error :   tf.Tensor(
[[1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1.
  1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0.
  1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1.
  0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1.
  1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1.
  1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1.
  0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
  1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0.
  1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0.
  1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1.
  1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1.
  0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1.
  0. 1. 0. 0. 1.

In [24]:
#Parameters
alpha = 0.01
epochs = 1
batchsize = 200
weights = []
errors = []
batch_number = 0
K = 1

In [26]:
#creating datasets
train_ds = \
  tf.data.Dataset.from_tensor_slices((trX , trY)).batch(batchsize)

for epoch in range (epochs):
  for batch_X , batch_Y in train_ds:
    batch_number += 1 
    for i_Sample in range(batchsize):
      for k in range(K):
        visi = batch_X[i_Sample]
        hid_State = hidden_layer(visi , W , hb)
        v1_State = reconstructed_output(hid_State , W , vb)
        h1_State = hidden_layer(v1_State , W , hb)

delta_W = tf.matmul(tf.transpose([visi]) , hid_State) - tf.matmul(tf.transpose([v1_State]), h1_State)
W = W + alpha* delta_W
vb = vb + alpha*tf.reduce_mean(visi - v1_State, 0 )
hb = hb + alpha*tf.reduce_mean(hid_State - h1_State , 0)
visi = v1_State
if i_sample == batchsize-1:
                err = error(batch_x[i_sample], v1_state)
                errors.append(err)
                weights.append(W)
                print ( 'Epoch: %d' % epoch, "batch #: %i " % batch_number, "of %i" % 60e3/batchsize /n)
                        


InvalidArgumentError: ignored

In [ ]:
#Parameters
alpha = 0.01
epochs = 1
batchsize = 200
weights = []
errors = []
batch_number = 0
K = 1

#creating datasets
train_ds = \
    tf.data.Dataset.from_tensor_slices((trX, trY)).batch(batchsize)

for epoch in range(epochs):
    for batch_x, batch_y in train_ds:
        batch_number += 1
        for i_sample in range(batchsize):           
            for k in range(K):
                v0_state = batch_x[i_sample]
                h0_state = hidden_layer(v0_state, W, hb)
                v1_state = reconstructed_output(h0_state, W, vb)
                h1_state = hidden_layer(v1_state, W, hb)

                delta_W = tf.matmul(tf.transpose([v0_state]), h0_state) - tf.matmul(tf.transpose([v1_state]), h1_state)
                W = W + alpha * delta_W

                vb = vb + alpha * tf.reduce_mean(v0_state - v1_state, 0)
                hb = hb + alpha * tf.reduce_mean(h0_state - h1_state, 0) 

                v0_state = v1_state

            if i_sample == batchsize-1:
                err = error(batch_x[i_sample], v1_state)
                errors.append(err)
                weights.append(W)
                print ( 'Epoch: %d' % epoch)



In [ ]:
 if i_sample == batchsize-1:
                err = error(batch_x[i_sample], v1_state)
                errors.append(err)
                weights.append(W)
                print ( 'Epoch: %d' % epoch, 
                       "batch #: %i " % batch_number, "of %i" % 60e3/batchsize 
                       "sample #: %i" % i_sample,
                       'reconstruction error: %f' % err)
